<a href="https://colab.research.google.com/github/skytecat/DLS/blob/main/homework_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Детекция объектов</b></h3>

В этом домашнем задании мы продолжим работу над детектором из семинара, поэтому при необходимости можете заимствовать оттуда любой код.

Домашнее задание можно разделить на следующие части:

* Переделываем модель [4]
  * Backbone[1],
  * Neck [2],
  * Head [1]
* Label assignment [3]:
  * TAL [3]
* Лоссы [1]:
  * CIoU loss [1]
* Кто больше? [5]
  * 0.05 mAP [1]
  * 0.1 mAP  [2]
  * 0.2 mAP [5]

**Максимальный балл:** 10 баллов. (+3 балла бонус).

In [34]:
import torch
import numpy as np
import pandas as pd
import albumentations as A

from PIL import Image
import io
from torchvision import transforms
from torch.utils.data import Dataset
from albumentations.pytorch.transforms import ToTensorV2

### Загрузка данных

Мы продолжаем работу с датасетом из семинара - Halo infinite ([сслыка](https://universe.roboflow.com/graham-doerksen/halo-infinite-angel-aim)). Загрузка данных и создание датасета полностью скопированы из семинара.

Сначала загружаем данные

In [2]:
splits = {'train': 'data/train-00000-of-00001-0d6632d599c29801.parquet',
          'validation': 'data/validation-00000-of-00001-c6b77a557eeedd52.parquet',
          'test': 'data/test-00000-of-00001-866d29d8989ea915.parquet'}
df_train = pd.read_parquet("hf://datasets/Francesco/halo-infinite-angel-videogame/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/Francesco/halo-infinite-angel-videogame/" + splits["test"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Создаем датасет для предобработки данных

In [38]:
import torch
import numpy as np
from PIL import Image
import io

class HaloDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data  # pandas DataFrame

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # --- Изображение ---
        image = Image.open(io.BytesIO(row["bytes"]))
        image = np.array(image)
        # Переводим в [C, H, W] и нормализуем в [0, 1]
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

        # --- Боксы ---
        boxes = row["boxes"]  # ожидается: список или массив [[x1,y1,x2,y2], ...]
        if isinstance(boxes, list):
            boxes = np.array(boxes, dtype=np.float32)
        boxes = torch.from_numpy(boxes).float()  # [N, 4]

        # --- Метки ---
        labels = row["labels"]  # ожидается: список или массив [1, 3, ...]
        if isinstance(labels, list):
            labels = np.array(labels, dtype=np.int64)
        labels = torch.from_numpy(labels).long()  # [N]

        # --- Проверка ---
        assert boxes.ndim == 2 and boxes.shape[1] == 4, f"boxes shape: {boxes.shape}"
        assert labels.ndim == 1, f"labels shape: {labels.shape}"
        assert len(boxes) == len(labels), f"boxes ({len(boxes)}) != labels ({len(labels)})"

        target = {
            'boxes': boxes,
            'labels': labels
        }

        return image, target

In [48]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [46]:
# class HaloDataset(Dataset):
#     def __init__(self, dataframe, transform=None):
#         df_objects = pd.json_normalize(dataframe['objects'])[["bbox", "category"]]
#         df_images = pd.json_normalize(dataframe['image'])[["bytes"]]
#         self.data = dataframe[["image_id"]].join(df_objects).join(df_images)
#         self.transform = transform

#     def __len__(self):
#         return len(self.data)
#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]

#         # Изображение
#         image = Image.open(io.BytesIO(row["bytes"]))
#         image = np.array(image)
#         image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0  # [C, H, W], float

#         # Боксы (должны быть в формате XYXY, float32)
#         boxes = torch.from_numpy(row["boxes"]).float()  # shape: [N, 4]

#         # Метки классов (должны быть long, и классы с 1, не с 0!)
#         labels = torch.from_numpy(row["labels"]).long()  # shape: [N]

#         target = {
#             'boxes': boxes,
#             'labels': labels
#         }

#         return image, target

#     # def __getitem__(self, idx):
#     #     """Загружаем данные и разметку для объекта с индексом `idx`.

#     #     labels: List[int] Набор классов для каждого ббокса,
#     #     boxes: List[List[int]] Набор ббоксов в формате (x_min, y_min, w, h).
#     #     """
#     #     row = self.data.iloc[idx]
#     #     image = Image.open(io.BytesIO(row["bytes"]))
#     #     image = np.array(image)

#     #     target = {}
#     #     target["image_id"] = row["image_id"]

#     #     labels = [row["category"]] if isinstance(row["category"], int) else row['category']
#     #     # Вычитаем единицу чтобы классы начинались с нуля
#     #     labels = [label - 1 for label in labels]
#     #     boxes = row['bbox'].tolist()

#     #     if self.transform is not None:
#     #         transformed = self.transform(image=image, bboxes=boxes, labels=labels)
#     #         image, boxes, labels = transformed["image"], transformed["bboxes"], transformed["labels"]
#     #     else:
#     #         image = transforms.ToTensor()(image)

#     #     target['boxes'] = torch.tensor(np.array(boxes), dtype=torch.float32)
#     #     target['labels'] = torch.tensor(labels, dtype=torch.int64)
#     #     return image, target

# def collate_fn(batch):
#     batch = tuple(zip(*batch))
#     images = torch.stack(batch[0])
#     return images, batch[1]

Чтобы модель не переобучалась, можно добавить больше аугментаций, весь список можно посмотреть тут [[ссылка](https://explore.albumentations.ai/)].

Какие можно использовать аугментации?
* Добавить зум `RandomResizedCrop`,
* Сделать цветовые аугментации типа `RandomBrightnessContrast` и/или `HueSaturationValue`,
* Добавить шум `GaussNoise`,
* Вырезать случайные части изображения `CoarseDropout`,
* И любые другие!

Аугментации можно комбинировать посредствам `A.OneOf`, `A.SomeOf` или `A.RandomOrder`.

Хоть аугментации ограничиваются только вашей фантазией, перед обучением советуем посмотреть на результат преобразований и убедиться, что изображение ещё поддается детекции:)

In [4]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.Normalize(mean=mean, std=std),
        ToTensorV2(),
    ],
    # Раскомментируй, если аугментации изменяют ббоксы.
    # Не забудь указать верный формат для ббоксов.
    bbox_params=A.BboxParams(format='coco', label_fields=['labels'])
)

test_transform = A.Compose(
    [
        A.Normalize(mean=mean, std=std),
        ToTensorV2(),
    ]
)

Не забываем инициализировать наш датасет

In [5]:
train_dataset = HaloDataset(df_train, transform=train_transform)
test_dataset = HaloDataset(df_test, transform=test_transform)

## Переделываем модель [4 балла]

В семинаре мы реализовали самый базовый детектор, а сейчас настало время его улучшать.

### Backbone [1 балл]

Хорошей практикой считается размораживать несколько последних слоев в backbone, это позволяет немного улучить качество модели. Давайте улушчим класс Backbone из лекции, добавив ему возможность разморозки __k__ последних слоев или блоков (на ваш выбор).

In [27]:
import torch.nn as nn
import timm

class Backbone(nn.Module):
    def __init__(self, model_name="efficientnet_b0", out_indices=(-1, -2, -3), unfreeze_last_k=0):
        super().__init__()
        # timm.list_models(pretrained=True)
        self.backbone = timm.create_model(model_name, pretrained=True, features_only=True, out_indices=out_indices)
        # Заморозим все
        for param in self.backbone.parameters():
            param.requires_grad = False
        # Разморозим k последних блоков
        if unfreeze_last_k > 0:
          blocks = self.backbone.blocks
          for block in blocks[-unfreeze_last_k:]:
            for param in block.parameters():
              param.requires_grad = True

    def forward(self, x):
        return self.backbone(x)

### NECK [2 балла]

Следующее улучшение коснется шеи. Предлагаем реализовать знакомую из лекции архитектуру FPN.

#### Feature Pyramid Network

<center><img src="https://user-images.githubusercontent.com/57972646/69858594-b14a6c00-12d5-11ea-8c3e-3c17063110d3.png"/></center>


* [Feature Pyramid Networks for Object Detection](https://arxiv.org/abs/1612.03144)

Она состоит из top-down пути, в котором происходит 2 вещи:
1. Увеличивается пространственная размерность фичей,
2. С помощью скипконнекшеннов, добавляются фичи из backbone модели.

Для увеличения пространственной размерности используется __nearest neighbor upsampling__, а фичи из шеи и бекбоуна суммируются.

__TIPS__:
* Можете использовать базовые классы из лекции,
* Воспользуйтесь AnchorGenerator-ом, чтобы создавать якоря сразу для нескольких выходов,
* Не забудьте использовать nn.ModuleList, если захотите сделать динамическое количество голов у модели,
* Также, можно добавить доп конволюцию (3х3 с паддингом) у каждого выхода шеи.

In [29]:
# import torch
# import torch.nn as nn
import torch.nn.functional as F

class Neck(nn.Module):
    def __init__(self, in_channels_list, out_channels=256):
        """

        Args:
            in_channels_list (List[int]): число каналов на выходе backbone для каждого уровня
                                          Ожидается порядок от ГЛУБОКИХ к МЕЛКИМ признакам:
                                          например, [2048, 1024, 512] для ResNet50 с out_indices=(-1, -2, -3)
            out_channels (int): число каналов на выходе каждого уровня FPN
        """
        super().__init__()

        self.out_channels = out_channels
        # Мы будем обрабатывать уровни от глубоких к мелким (C5 → C4 → C3)
        # lateral_convs: 1x1 свёртки для выравнивания каналов
        self.lateral_convs = nn.ModuleList()
        # output_convs: 3x3 свёртки для сглаживания
        self.output_convs = nn.ModuleList()

        for in_channels in in_channels_list:
            self.lateral_convs.append(
                nn.Conv2d(in_channels, out_channels, kernel_size=1)
            )
            self.output_convs.append(
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
            )

    def forward(self, features):
        """
        Args:
            features (List[Tensor]): признаки от backbone'а.
                                     Порядок: [C5, C4, C3] — от глубоких к мелким
        Returns:
            fpn_features (List[Tensor]): [P3, P4, P5] — от мелких к глубоким
        """
        # 1. Применяем lateral 1x1 свёртки
        lateral_features = []
        for i, feat in enumerate(features):
            lateral_features.append(self.lateral_convs[i](feat))

        # 2. Top-down путь: начинаем с самого глубокого уровня
        # P5 = lateral(C5)
        fpn_features = [lateral_features[0]]

        # Идём от P5 к P4, P3, ...
        for i in range(1, len(lateral_features)):
            # Апсемплинг предыдущего уровня до размера текущего
            upsampled = F.interpolate(
                fpn_features[-1],
                size=lateral_features[i].shape[-2:],  # (H, W)
                mode='nearest'
            )
            # Суммируем с lateral feature из backbone'а
            fpn = upsampled + lateral_features[i]
            fpn_features.append(fpn)

        # 3. Применяем post-processing 3x3 свёртки
        output_features = []
        for i, fpn in enumerate(fpn_features):
            out = self.output_convs[i](fpn)
            output_features.append(out)

        # 4. Возвращаем в порядке от МЕЛКИХ к ГЛУБОКИМ: [P3, P4, P5]
        return output_features[::-1]

### Head [1 балл]

В качестве шеи можно выбрать __один из двух__ вариантов:

#### 1. Decoupled Head

Реализовать Decoupled Head из [YOLOX](https://arxiv.org/abs/2107.08430).
<center><img src="https://i.ibb.co/BVtBR2R3/Decoupled-head.jpg"/></center>

**TIP**: Возьмите за основу голову из семинара, тк она сильно похожа на Decoupled Head.

Изменять количество параметров у шей на разных уровнях не обязательно.

#### 2. Confidence score free head

Нужно взять за основу голову из семинара и полностью убрать предсказание confidence score. Чтобы модель предсказывала только 2 группы: ббоксы и классы.

Есть следующие способы удаления confidence score:
* Добавление нового класса ФОН. Обычно его обозначают нулевым классом.
* Присваивание ббоксам БЕЗ объекта вектор из нулей в качестве таргета.

Выберете тот, который вам больше нравится и будте внимательны при расчете лосса!

**Важно!** Удаление confidence score повлияет на следующие методы из семинара:
* target_assign
* ComputeLoss
* _filter_predictions

In [8]:
class Head(nn.Module):
    def __init__(self, in_channels, num_anchors, num_classes):
        super().__init__()
        self.num_anchors = num_anchors
        self.num_classes = num_classes

        self.stem = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)

        # Отдельные ветки
        # Ветка классификации
        self.cls_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.cls_bn = nn.BatchNorm2d(in_channels)
        self.cls_head = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=1)

        # Ветка регрессии + confidence
        self.reg_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.reg_bn = nn.BatchNorm2d(in_channels)
        self.reg_head = nn.Conv2d(in_channels, num_anchors * 5, kernel_size=1)  # 4 coord + 1 obj

    def forward(self, x):
        x = F.relu(self.stem(x))

        # Classification branch
        cls_feat = F.relu(self.cls_bn(self.cls_conv(x)))
        cls_logits = self.cls_head(cls_feat)

        # Regression + Objectness branch
        reg_feat = F.relu(self.reg_bn(self.reg_conv(x)))
        bbox_and_obj = self.reg_head(reg_feat)  # [B, A*5, H, W]

        return cls_logits, bbox_and_obj

Теперь можно снова реализовать класс детектора с учетом всех частей выше!

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.detection.anchor_utils import AnchorGenerator

class Detector(nn.Module):
    """
    Детектор с FPN и Decoupled Head.

    Параметры
    ---------
    backbone_model_name : str
        Имя timm модели для backbone.
    neck_n_channels : int
        Число каналов на выходе шеи (FPN).
    num_classes : int
        Число объектных классов (без фона!).
    anchor_sizes : tuple of tuples
        Размеры якорей для КАЖДОГО уровня FPN, например: ((32,), (64,), (128,))
    anchor_ratios : tuple of tuples
        Пропорции якорей для КАЖДОГО уровня, например: ((0.5, 1.0, 2.0),) * 3
    input_size : tuple
        Размер входного изображения (H, W).
    unfreeze_last_k : int
        Сколько последних блоков backbone'а разморозить.
    """
    def __init__(self,
                 backbone_model_name="efficientnet_b0",
                 neck_n_channels=256,
                 num_classes=4,
                 anchor_sizes=((32,), (64,), (128,)),
                 anchor_ratios=((0.5, 1.0, 2.0), (0.5, 1.0, 2.0), (0.5, 1.0, 2.0)),
                 input_size=(640, 640),
                 unfreeze_last_k=0
        ):
        super().__init__()
        self.num_classes = num_classes
        self.input_size = input_size

        # === 1. BACKBONE (с несколькими выходами) ===
        # В FPN обычно берут 3-4 уровня: out_indices=(-1, -2, -3)
        self.backbone = Backbone(
            model_name=backbone_model_name,
            out_indices=(-1, -2, -3),
            unfreeze_last_k=unfreeze_last_k
        )

        # Получаем число каналов каждого уровня
        dummy = torch.randn(1, 3, input_size[0], input_size[1])
        with torch.no_grad():
            feats = self.backbone(dummy)
            in_channels_list = [f.shape[1] for f in feats]  # [C5, C4, C3]

        # === 2. NECK (FPN) ===
        self.neck = Neck(in_channels_list=in_channels_list, out_channels=neck_n_channels)

        # === 3. HEADS (по одному на уровень FPN) ===
        num_levels = len(in_channels_list)
        self.heads = nn.ModuleList([
            Head(
                in_channels=neck_n_channels,
                num_anchors=len(anchor_sizes[i]) * len(anchor_ratios[i]),
                num_classes=num_classes
            )
            for i in range(num_levels)
        ])

        # === 4. ANCHOR GENERATOR ===
        self.anchor_generator = AnchorGenerator(
            sizes=anchor_sizes,
            aspect_ratios=anchor_ratios
        )

        # Предвычисляем якоря для фиксированного input_size
        self._precompute_anchors(input_size)

    def _precompute_anchors(self, input_size):
        """Предвычисляет якоря для заданного размера изображения."""
        # Получаем размеры feature maps после backbone + neck
        dummy = torch.randn(1, 3, input_size[0], input_size[1])
        with torch.no_grad():
            backbone_feats = self.backbone(dummy)
            fpn_feats = self.neck(backbone_feats)  # [P3, P4, P5]
            grid_sizes = [f.shape[-2:] for f in fpn_feats]  # [(H3, W3), (H4, W4), (H5, W5)]

        # Вычисляем strides (на сколько уменьшено изображение)
        strides = [
            [input_size[0] // h, input_size[1] // w]
            for h, w in grid_sizes
        ]

        # Генерируем якоря
        anchors_list = self.anchor_generator.grid_anchors(grid_sizes, strides)
        anchors = torch.cat(anchors_list, dim=0)  # [N_total_anchors, 4]

        # Сохраняем в буфере (чтобы не пересчитывать каждый раз)
        self.register_buffer("anchors", anchors)
        self.register_buffer("anchor_centers", (anchors[:, :2] + anchors[:, 2:]) / 2)
        self.register_buffer("anchor_sizes", anchors[:, 2:] - anchors[:, :2])

    def forward(self, x):
        """
        Args:
            x: тензор [B, C, H, W]
        Returns:
            В зависимости от режима:
            - training: списки предсказаний по уровням,
            - eval: декодированные боксы и вероятности.
        """
        # 1. Backbone
        backbone_feats = self.backbone(x)  # [C5, C4, C3]

        # 2. Neck (FPN)
        fpn_feats = self.neck(backbone_feats)  # [P3, P4, P5]

        # 3. Heads
        all_cls_logits = []
        all_bbox_preds = []
        all_confidence_logits = []

        for i, feat in enumerate(fpn_feats):
            cls_logits, bbox_and_obj = self.heads[i](feat)
            # Разделяем bbox и confidence
            bbox_offsets = bbox_and_obj[:, :4 * self.heads[i].num_anchors, :, :]
            confidence_logits = bbox_and_obj[:, 4 * self.heads[i].num_anchors:, :, :]

            all_cls_logits.append(cls_logits)
            all_bbox_preds.append(bbox_offsets)
            all_confidence_logits.append(confidence_logits)

        if self.training:
            # Возвращаем "сырые" предсказания по уровням
            return {
                'cls_logits': all_cls_logits,
                'bbox_offsets': all_bbox_preds,
                'confidence_logits': all_confidence_logits
            }

        # === Inference: декодируем все боксы ===
        all_bboxes = []
        all_confidences = []
        all_cls_probs = []

        start_idx = 0
        for i, feat in enumerate(fpn_feats):
            B, _, H, W = feat.shape
            num_anchors_level = self.heads[i].num_anchors
            num_preds = H * W * num_anchors_level

            # Берём смещения для этого уровня
            bbox_offsets_flat = all_bbox_preds[i].permute(0, 2, 3, 1).contiguous().view(B, -1, 4)
            conf_flat = all_confidence_logits[i].permute(0, 2, 3, 1).contiguous().view(B, -1)
            cls_flat = all_cls_logits[i].permute(0, 2, 3, 1).contiguous().view(B, -1, self.num_classes)

            # Декодируем боксы
            anchors_level = self.anchors[start_idx:start_idx + num_preds]
            bboxes = self._decode_level(bbox_offsets_flat, anchors_level)

            all_bboxes.append(bboxes)
            all_confidences.append(torch.sigmoid(conf_flat))
            all_cls_probs.append(torch.softmax(cls_flat, dim=-1))

            start_idx += num_preds

        # Объединяем все уровни
        final_bboxes = torch.cat(all_bboxes, dim=1)
        final_confidences = torch.cat(all_confidences, dim=1)
        final_cls_probs = torch.cat(all_cls_probs, dim=1)

        return final_bboxes, final_confidences, final_cls_probs

    def _decode_level(self, bbox_offsets, anchors):
        """Декодирует боксы для одного уровня."""
        tx = bbox_offsets[:, :, 0]
        ty = bbox_offsets[:, :, 1]
        tw = bbox_offsets[:, :, 2]
        th = bbox_offsets[:, :, 3]

        anchor_centers = (anchors[:, :2] + anchors[:, 2:]) / 2
        anchor_wh = anchors[:, 2:] - anchors[:, :2]

        cx = anchor_centers[:, 0] + torch.sigmoid(tx) * anchor_wh[:, 0]
        cy = anchor_centers[:, 1] + torch.sigmoid(ty) * anchor_wh[:, 1]
        w = torch.exp(tw) * anchor_wh[:, 0]
        h = torch.exp(th) * anchor_wh[:, 1]

        x_min = cx - w / 2
        y_min = cy - h / 2
        return torch.stack([x_min, y_min, w, h], dim=-1)

## Label assignment [3 балла]
В этой секции предлагается заменить функцию `assign_target` на более современный алгоритм который называется Task alignment learning.

Он описан в статье [TOOD](https://arxiv.org/abs/2108.07755) в секции 3.2. Для удобства вот его основные шаги:

1. Посчитать значение метрики для каждого предсказанного ббокса:
    
$$t = s^\alpha * u^\beta$$
    
где,
* $s$ — classification score, или вероятность принадлежности предсказанного ббокса к классу реального ббокса (**GT**);
* $u$ — IoU между предсказанным и реальным ббоксами;
* $\alpha,\ \beta$ — нормализационные константы, обычно $\alpha = 6.0, \ \beta = 1.0$.
    
2. Отфильтровать предсказания на основе **GT**.

    Для якорных детекторов, обычно, выбираются только те предсказания, центры якорей которых находятся внутри GT.
4. Для каждого **GT** выбрать несколько (обычно 5 или 13) самых подходящих предсказаний.
5. Если предсказание рассматривается в качестве подходящего для нескольких **GT** — выбрать **GT** с наибольшим пересечением по IoU.


**BAЖНО**: если будете использовать Runner из лекции, не забудьте поменять параметры  в `self.assign_target_method` в методе `_run_train_epoch`.

In [10]:
import torch
import torch.nn.functional as F

def bbox_iou(box1, box2, eps=1e-7):
    """Вычисляет IoU между двумя наборами боксов.

    Args:
        box1: [N, 4] в формате (x1, y1, x2, y2)
        box2: [M, 4] в формате (x1, y1, x2, y2)
    Returns:
        iou: [N, M]
    """
    # Площади
    area1 = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
    area2 = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])

    # Пересечение
    lt = torch.max(box1[:, None, :2], box2[:, :2])  # [N, M, 2]
    rb = torch.min(box1[:, None, 2:], box2[:, 2:])  # [N, M, 2]

    wh = (rb - lt).clamp(min=0)  # [N, M, 2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N, M]

    iou = inter / (area1[:, None] + area2 - inter + eps)
    return iou

def TAL_assigner(
    anchors,          # [num_anchors, 4] — все якоря в (x1, y1, x2, y2)
    gt_boxes,         # [num_gts, 4] — GT боксы в (x1, y1, x2, y2)
    gt_labels,        # [num_gts] — метки классов (0 не используется, классы с 1)
    cls_probs,        # [num_anchors, num_classes] — вероятности классов (после sigmoid/softmax)
    pred_boxes,       # [num_anchors, 4] — предсказанные боксы (x1, y1, x2, y2)
    topk=13,          # сколько якорей выбрать на GT
    alpha=6.0,
    beta=1.0,
    device="cpu"
):
    """
    Task Alignment Learning (TAL) assigner из TOOD.

    Returns:
        pos_mask: [num_anchors] — булев маска положительных якорей
        assigned_gt_inds: [num_anchors] — индекс GT для каждого якоря (-1 = фон)
        assigned_labels: [num_anchors] — метка класса для каждого якоря (0 = фон)
    """
    num_anchors = anchors.size(0)
    num_gts = gt_boxes.size(0)

    if num_gts == 0:
        return (
            torch.zeros(num_anchors, dtype=torch.bool, device=device),
            torch.full((num_anchors,), -1, dtype=torch.long, device=device),
            torch.zeros(num_anchors, dtype=torch.long, device=device)
        )

    # 1. Фильтрация: якоря, центры которых внутри GT
    anchor_centers = (anchors[:, :2] + anchors[:, 2:]) / 2  # [num_anchors, 2]
    # Проверяем, что центр якоря внутри хотя бы одного GT
    is_anchor_in_gt = (
        (anchor_centers[:, 0] >= gt_boxes[:, 0].min()) &
        (anchor_centers[:, 0] <= gt_boxes[:, 2].max()) &
        (anchor_centers[:, 1] >= gt_boxes[:, 1].min()) &
        (anchor_centers[:, 1] <= gt_boxes[:, 3].max())
    )  # Это упрощение! Лучше по каждому GT.

    # Более точная проверка: центр якоря внутри КАЖДОГО GT
    is_anchor_in_gt = (
        (anchor_centers[:, None, 0] >= gt_boxes[:, 0]) &  # x >= x1
        (anchor_centers[:, None, 0] <= gt_boxes[:, 2]) &  # x <= x2
        (anchor_centers[:, None, 1] >= gt_boxes[:, 1]) &  # y >= y1
        (anchor_centers[:, None, 1] <= gt_boxes[:, 3])    # y <= y2
    )  # [num_anchors, num_gts]

    # 2. Вычисляем IoU между предсказанными боксами и GT
    ious = bbox_iou(pred_boxes, gt_boxes)  # [num_anchors, num_gts]

    # 3. Получаем classification score для каждого (якорь, GT)
    # cls_probs: [num_anchors, num_classes]
    # gt_labels: [num_gts] — метки от 1 до num_classes
    cls_scores = cls_probs[torch.arange(num_anchors)[:, None], gt_labels]  # [num_anchors, num_gts]

    # 4. Вычисляем TAL score: t = s^α * u^β
    alignment_metrics = (cls_scores ** alpha) * (ious ** beta)  # [num_anchors, num_gts]

    # 5. Применяем маску: только якоря, центры которых внутри GT
    alignment_metrics = alignment_metrics * is_anchor_in_gt.float()  # [num_anchors, num_gts]

    # 6. Для каждого GT выбираем topk якорей по alignment_metrics
    # topk_anchors: [num_gts, topk]
    topk_metrics, topk_indices = torch.topk(
        alignment_metrics.T,  # [num_gts, num_anchors]
        k=min(topk, num_anchors),
        dim=1,
        largest=True
    )

    # 7. Назначаем GT каждому якорю
    assigned_gt_inds = torch.full((num_anchors,), -1, dtype=torch.long, device=device)
    assigned_labels = torch.zeros(num_anchors, dtype=torch.long, device=device)

    # Для каждого GT
    for gt_idx in range(num_gts):
        anchor_indices = topk_indices[gt_idx]  # [topk]
        # Назначаем GT этим якорям
        for anchor_idx in anchor_indices:
            # Если якорь уже назначен другому GT — выбираем по IoU
            if assigned_gt_inds[anchor_idx] != -1:
                # Сравниваем IoU с текущим и предыдущим GT
                current_iou = ious[anchor_idx, gt_idx]
                prev_gt = assigned_gt_inds[anchor_idx]
                prev_iou = ious[anchor_idx, prev_gt]
                if current_iou > prev_iou:
                    assigned_gt_inds[anchor_idx] = gt_idx
                    assigned_labels[anchor_idx] = gt_labels[gt_idx]
            else:
                assigned_gt_inds[anchor_idx] = gt_idx
                assigned_labels[anchor_idx] = gt_labels[gt_idx]

    pos_mask = assigned_gt_inds != -1

    return pos_mask, assigned_gt_inds, assigned_labels

### DIoU [1]

Вместо SmoothL1, который используется в семинаре, реализуем лосс, основанный на пересечении ббоксов. В качестве тренировки давайте напишем Distance Intersection over Union (DIoU).

<center><img src=https://wikidocs.net/images/page/163613/Free_Fig_5.png></center>

Для его реализации разобъем задачу на части:

**1. Реализуем IoU:**

Пусть даны координаты для предсказанного ($B^p$) и истинного ($B^g$) ббоксов в формате XYXY или VOC PASCAL (левый верхний и правый нижний углы):

$B^p=(x^p_1, y^p_1, x^p_2, y^p_2)$, $B^g=(x^g_1, y^g_1, x^g_2, y^g_2)$, тогда алгоритм расчета будет следующий:

    1. Найдем площади обоих ббоксов:
$$ A^p = (x^p_2 - x^p_1) * (y^p_2 - y^p_1) $$
$$ A^g = (x^g_2 - x^g_1) * (y^g_2 - y^g_1) $$

    2. Посчитаем пересечение между ббоксами:

Тут мы предлагаем вам подумать как в общем виде можно расчитать размеры ббокса, который будет являться пересечением $B^p$ и $B^g$, а затем посчитать его площадь:

$$x^I_1 = \qquad \qquad y^I_1 = $$
$$x^I_2 = \qquad \qquad y^I_2 = $$

В общем виде, площать будет записываться следующим образом:

Если $x^I_2 > x^I_1$ & $y^I_2 > y^I_1$, тогда:

$$I = (x^I_2 - x^I_1) * (y^I_2 - y^I_1)$$

Иначе, $I = 0$.

    3. Считаем объединение ббоксов.

Мы можем посчитать эту площадь как сумму площадей двух ббоксов минус площадь пересечения (тк мы считаем её два раз в сумме площадей):

$$U = A^p + A^g - I$$

    4. Вычисляем IoU.

$$IoU = \frac{I}{U}$$

**2. Посчитаем диагональ выпуклой оболочки:**

Для расчета диагонали, сначала выпишите координаты верхнего левого и правого нижнего углов. Подумайте, чему будут равны эти координаты в общем случае?

$$x^c_1 = \qquad \qquad y^c_1 = $$
$$x^c_2 = \qquad \qquad y^c_2 = $$

Подсказка: Нарисуйте несколько вариантов пересечений предсказания и GT на бумажке, и выпишите координаты для выпуклой оболочки.

Тогда квадрат диагонали можно посчитать по формуле:

$$c^2 = (x^c_2 - x^c_1)^2 + (y^c_2 - y^c_1)^2$$

**3. Рассчитаем расстояние между цетрами ббоксов:**

Сначала находим координаты центров каждого из ббоксов (если ббоксы в формате YOLO, то и считать ничего не нужно), затем считаем Евклидово расстояние между центрами.

$d = $

Собираем все части вместе и считаем лосс по формуле:

$$ DIoU = 1 - IoU + \frac{d^2}{c^2}$$

Помните, что пар ббоксов может быть много! Возвращайте усредненное значение лосса.

In [11]:
from torchvision.ops import distance_box_iou_loss

In [12]:
def gen_bbox(num_boxes=10):
    min_corner = torch.randint(0, 100, (num_boxes, 2))
    max_corner = torch.randint(50, 150, (num_boxes, 2))

    for i in range(2):
        wrong_order = min_corner[:, i] > max_corner[:, i]
        if wrong_order.any():
            min_corner[wrong_order, i], max_corner[wrong_order, i] = max_corner[wrong_order, i], min_corner[wrong_order, i]
    return torch.cat((min_corner, max_corner), dim=1)

In [13]:
pred_boxes = gen_bbox(num_boxes=100)
true_boxes = gen_bbox(num_boxes=100)

In [14]:
print(f" DIoU: {distance_box_iou_loss(pred_boxes, true_boxes, reduction="mean").item()}")

 DIoU: 1.0210096836090088


In [15]:
import torch
import torch.nn.functional as F

def diou_loss(pred_boxes, gt_boxes):
    """
    DIoU Loss для боксов в формате XYXY (x1, y1, x2, y2).

    Args:
        pred_boxes: [N, 4] — предсказанные боксы
        gt_boxes:   [N, 4] — истинные боксы

    Returns:
        mean DIoU loss (scalar)
    """
    # 1. Площади предсказанных и истинных боксов
    pred_w = pred_boxes[:, 2] - pred_boxes[:, 0]
    pred_h = pred_boxes[:, 3] - pred_boxes[:, 1]
    gt_w = gt_boxes[:, 2] - gt_boxes[:, 0]
    gt_h = gt_boxes[:, 3] - gt_boxes[:, 1]

    pred_area = pred_w * pred_h  # [N]
    gt_area = gt_w * gt_h        # [N]

    # 2. Площадь пересечения
    # Координаты пересечения
    xI1 = torch.max(pred_boxes[:, 0], gt_boxes[:, 0])
    yI1 = torch.max(pred_boxes[:, 1], gt_boxes[:, 1])
    xI2 = torch.min(pred_boxes[:, 2], gt_boxes[:, 2])
    yI2 = torch.min(pred_boxes[:, 3], gt_boxes[:, 3])

    # Размеры пересечения
    inter_w = (xI2 - xI1).clamp(min=0)
    inter_h = (yI2 - yI1).clamp(min=0)
    inter_area = inter_w * inter_h  # [N]

    # 3. Объединение
    union_area = pred_area + gt_area - inter_area  # [N]

    # 4. IoU
    iou = inter_area / (union_area + 1e-7)  # [N]

    # 5. Диагональ наименьшего охватывающего прямоугольника
    xc1 = torch.min(pred_boxes[:, 0], gt_boxes[:, 0])
    yc1 = torch.min(pred_boxes[:, 1], gt_boxes[:, 1])
    xc2 = torch.max(pred_boxes[:, 2], gt_boxes[:, 2])
    yc2 = torch.max(pred_boxes[:, 3], gt_boxes[:, 3])

    c_sq = (xc2 - xc1) ** 2 + (yc2 - yc1) ** 2  # [N]

    # 6. Расстояние между центрами
    pred_center_x = (pred_boxes[:, 0] + pred_boxes[:, 2]) / 2
    pred_center_y = (pred_boxes[:, 1] + pred_boxes[:, 3]) / 2
    gt_center_x = (gt_boxes[:, 0] + gt_boxes[:, 2]) / 2
    gt_center_y = (gt_boxes[:, 1] + gt_boxes[:, 3]) / 2

    d_sq = (pred_center_x - gt_center_x) ** 2 + (pred_center_y - gt_center_y) ** 2  # [N]

    # 7. DIoU
    diou = 1 - iou + d_sq / (c_sq + 1e-7)  # [N]

    return diou.mean()

In [16]:
import numpy as np
pred_boxes = gen_bbox(num_boxes=1000)
true_boxes = gen_bbox(num_boxes=1000)

# проверим что написанный лосс выдает те же результаты что и лосс из торча.
assert np.isclose(diou_loss(pred_boxes, true_boxes), distance_box_iou_loss(pred_boxes, true_boxes, reduction="mean"))

## Кто больше? [5 баллов]

Наконец то мы дошли до самый интересной части. Тут мы раздаем очки за mAP'ы!

Все что вы написали выше вам поможет улучшить качество итогового детектора, настало время узнать насколько сильно :)

За достижения порога по mAP на тестовом наборе вы получаете баллы:
* 0.05 mAP [1]
* 0.1 mAP [2]
* 0.2 mAP [5]


**TIPS**:
1. На семинаре мы специально не унифицировали формат ббоксов между методами, чтобы обратить ваше внимание что за этим нужно следить. Чтобы было проще, сразу унифицируете формат по всему ноутбуку. Советуем использовать формат xyxy, тк IoU и NMS из torch используют именно этот формат. (Не забудьте поменять формат у таргета в `HaloDataset`).

2. Попробуйте перейти к IoU-based лоссу при обучении. То есть обучать не смещения, а сразу предсказывать ббокс.

3. Поэксперементируйте с подходами target assignment'а в процессе обучения. Например, можно на первых итерациях использовать обычный метод, а затем подключить TAL.

4. Добавьте аугментаций!

Можно взять [albumentations](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/), библиотеку, которую мы использовали всеминаре. Или базовые аугментации из торча [тык](https://pytorch.org/vision/main/transforms.html). Если будете использовать торч, не забудте про ббоксы, transforms из коробки не будет их агументировать.

5. Можете реализовать другую шею, которую мы обсуждали на лекции [Path Aggregation Network](https://arxiv.org/abs/1803.01534) она точно улучшит ваше итоговое качество.

6. Попробуйте добавлять различные блоки из YOLO архитектур в шею вместо единичных конволюционных слоев. (Например, замените конволюции 3х3 на CSP блоки).

7. Попробуйте заменить NMS на другой метод (WeightedNMS, SoftNMS, etc.). Немного ссылок:
    * Статья про SoftNMS [тык](https://arxiv.org/pdf/1704.04503)
    * Статья про WeightedNMS [тык](https://openaccess.thecvf.com/content_ICCV_2017_workshops/papers/w14/Zhou_CAD_Scale_Invariant_ICCV_2017_paper.pdf)
    * Есть их реализация, правда на нумбе [git](https://github.com/ZFTurbo/Weighted-Boxes-Fusion?tab=readme-ov-file)

8. Не бойтесь эксперементировать и удачи!

Также, напишите развернутые ответы на следующие вопросы:

**Questions:**
1. Какой метод label assignment'a помогает лучше обучаться модели? Почему?
2. Какое из сделаных вами улучшений внесло наибольший вклад в качество модели? Как вы думаете, почему это произошло?
3. Какое из сделанных вами улучшений вообще не изменило метрику? Как вы думаете, почему это произошло?

In [17]:
def filter_predictions(
    outputs,
    score_threshold=0.1,
    nms_threshold=0.5,
    device="cpu"
):
    """
    Преобразует сырые выходы модели в формат [{'boxes', 'scores', 'labels'}, ...]
    """
    all_predictions = []
    bboxes, confidences, cls_probs = outputs  # [B, N, 4], [B, N], [B, N, C]

    B = bboxes.shape[0]
    for i in range(B):
        # Фильтрация по confidence
        valid_mask = confidences[i] > score_threshold
        if valid_mask.sum() == 0:
            all_predictions.append({
                'boxes': torch.empty(0, 4, device=device),
                'scores': torch.empty(0, device=device),
                'labels': torch.empty(0, dtype=torch.long, device=device)
            })
            continue

        boxes_i = bboxes[i][valid_mask]
        scores_i = confidences[i][valid_mask]
        cls_probs_i = cls_probs[i][valid_mask]

        # Получаем метки и вероятности классов
        class_scores, class_labels = cls_probs_i.max(dim=1)
        final_scores = scores_i * class_scores  # TOOD-style: obj * cls

        # NMS
        keep = torch.ops.torchvision.nms(
            boxes_i, final_scores, nms_threshold
        )

        all_predictions.append({
            'boxes': boxes_i[keep],
            'scores': final_scores[keep],
            'labels': class_labels[keep]
        })

    return all_predictions


In [18]:
def detection_loss(
    model_outputs,
    targets,
    anchors,
    alpha=6.0,
    beta=1.0,
    topk=13,
    device="cpu"
):
    """
    Общий лосс: cls_loss + diou_loss + obj_loss
    """
    cls_logits_list = model_outputs['cls_logits']      # List[Tensor]
    bbox_offsets_list = model_outputs['bbox_offsets']  # List[Tensor]
    confidence_logits_list = model_outputs['confidence_logits']  # List[Tensor]

    # Объединяем все уровни в один тензор
    all_cls_logits = torch.cat([x.permute(0, 2, 3, 1).contiguous().view(-1, model_outputs['num_classes']) for x in cls_logits_list], dim=0)
    all_bbox_offsets = torch.cat([x.permute(0, 2, 3, 1).contiguous().view(-1, 4) for x in bbox_offsets_list], dim=0)
    all_conf_logits = torch.cat([x.permute(0, 2, 3, 1).contiguous().view(-1) for x in confidence_logits_list], dim=0)

    # Предсказанные боксы (в формате XYXY)
    pred_boxes = model.decode_bboxes(all_bbox_offsets)  # [N, 4]
    cls_probs = torch.sigmoid(all_cls_logits)           # [N, C]
    obj_probs = torch.sigmoid(all_conf_logits)          # [N]

    # TAL assigner
    gt_boxes = torch.cat([t['boxes'] for t in targets], dim=0)  # [M, 4]
    gt_labels = torch.cat([t['labels'] for t in targets], dim=0)  # [M]

    if gt_boxes.numel() == 0:
        # Нет объектов — лосс только на фон
        obj_loss = F.binary_cross_entropy_with_logits(all_conf_logits, torch.zeros_like(all_conf_logits))
        cls_loss = F.binary_cross_entropy_with_logits(all_cls_logits, torch.zeros_like(all_cls_logits))
        return obj_loss + cls_loss

    pos_mask, assigned_gt_inds, assigned_labels = TAL_assigner(
        anchors=anchors,
        gt_boxes=gt_boxes,
        gt_labels=gt_labels,
        cls_probs=cls_probs,
        pred_boxes=pred_boxes,
        topk=topk,
        alpha=alpha,
        beta=beta,
        device=device
    )

    # DIoU Loss только для положительных
    if pos_mask.any():
        diou_l = diou_loss(pred_boxes[pos_mask], gt_boxes[assigned_gt_inds[pos_mask]])
    else:
        diou_l = torch.tensor(0.0, device=device)

    # Objectness loss
    obj_targets = pos_mask.float()
    obj_loss = F.binary_cross_entropy_with_logits(all_conf_logits, obj_targets)

    # Classification loss (focal loss можно добавить)
    cls_targets = torch.zeros_like(cls_probs)
    if pos_mask.any():
        cls_targets[pos_mask, assigned_labels[pos_mask] - 1] = 1.0  # классы с 1, индексы с 0
    cls_loss = F.binary_cross_entropy_with_logits(all_cls_logits, cls_targets)

    return diou_l + obj_loss + cls_loss

In [19]:
@torch.no_grad()
def validate(model, dataloader, device="cpu", score_threshold=0.1, nms_threshold=0.5):
    model.eval()
    metric = MeanAveragePrecision(box_format="xyxy", iou_type="bbox")

    for images, targets in tqdm(dataloader, desc="Validation"):
        images = [img.to(device) for img in images]
        outputs = model(images)  # в eval режиме
        predicts = filter_predictions(
            outputs, score_threshold, nms_threshold, device=device
        )
        # targets уже в формате [{'boxes', 'labels'}, ...]
        metric.update(predicts, targets)

    return metric.compute()["map"].item()

In [23]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 13.6 MB/s eta 0:00:00


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchmetrics.detection import MeanAveragePrecision


def train_detector(
    model,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    anchors,
    device="cpu",
    num_epochs=30,
    patience=5,
    save_path="best_detector.pth"
):
    best_map = 0.0
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()
            outputs = model(images)  # в train режиме
            loss = detection_loss(outputs, targets, anchors, device=device)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        val_map = validate(model, val_loader, device=device)

        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Val mAP: {val_map:.4f}")

        # Сохраняем лучшую модель
        if val_map > best_map:
            best_map = val_map
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"  → New best mAP: {best_map:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping!")
                break

        if scheduler:
            scheduler.step()

    return best_map

In [49]:
# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Модель
model = Detector(
    backbone_model_name="efficientnet_b0",
    num_classes=4,
    anchor_sizes=((32,), (64,), (128,)),
    anchor_ratios=((0.5, 1.0, 2.0),) * 3,
    input_size=(640, 640),
    unfreeze_last_k=2
).to(device)

# Анкоры (предвычисленные)
anchors = model.anchors  # должен быть в XYXY

# Оптимизатор
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# Даталоадеры (должны возвращать images: List[Tensor], targets: List[Dict])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Обучение
best_map = train_detector(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    anchors=anchors,
    device=device,
    num_epochs=30,
    patience=5
)

print(f"Best mAP: {best_map:.4f}")

Epoch 1:   0%|          | 0/116 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'to'

Ниже определена вспомогательная функция для валидации качества. Можете использовать `Runner.validate`. Важное уточнение, ей нужен метод для фильтрации предсказаний. Можете тоже скопировать его из семинара, если он у вас не менялся.

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

@torch.no_grad()
def validate(dataloader, filter_predictions_func, box_format="xyxy", device="cpu", score_threshold=0.1, nms_threshold=0.5, **kwargs):
    """ Метод для валидации модели.
    Возвращает mAP (0.5 ... 0.95).
    """
    self.model.eval()
    # Считаем метрику mAP с помощью функции из torchmetrics
    metric = MeanAveragePrecision(box_format=box_format, iou_type="bbox")
    for images, targets in tqdm(dataloader, desc="Running validation", leave=False):
        images = images.to(device)
        outputs = self.model(images)
        predicts = filter_predictions_func(outputs, score_threshold, nms_threshold, **kwargs)
        metric.update(predicts, targets)
    return metric.compute()["map"].item()
